In [14]:
%run routeLinkedList.ipynb
# %run ../routeLinkedList.ipynb #Uncomment when testing

In [15]:
import requests
import math

REQUESTSTATES = {
    "NOT_CALLED": 0,
    "WAITING": 1,
    "SCHEDULED": 2,
    "IN_PROGRESS": 3,
    "COMPLETED": 4,
    "REJECTED": 5
    }

VEHICLESTATES = {
    "IDLE":0,
    "MOVING":1,
}

# OSRM instance details
OSRM_HOST = "localhost"  # ROSRM server IP or hostname
OSRM_PORT = "5000"       # Port of OSRM server is listening on

In [16]:
class Cords:
    def __init__(self, latidude, longitude):
        self.latidude = latidude
        self.longitude = longitude
    
    def __str__(self):
        return "lat: "+ str(self.latidude) + ",long: " + str(self.longitude)
    
    def getLatitude(self):
        return self.latidude
    
    def getLongitude(self):
        return self.longitude

In [17]:
class Request_Cords(Cords):
    def __init__(self, latidude, longitude, request_id, start):
        super().__init__(latidude, longitude)
        self.start = start
        self.request_id = request_id
    
    def __str__(self):
        return "Req_Id: " + str(self.request_id) + ", lat: "+ str(self.latidude) + ", long: " + str(self.longitude) + ", " + str(self.start)
    
    def __eq__(self, requestCord):
        if self.request_id == requestCord.getRequestId() and self.start == requestCord.getStart():
            return True
        else:
            return False
        
    def getLatitude(self):
        return self.latidude
    
    def getLongitude(self):
        return self.longitude
    
    def getStart(self):
        return self.start
    
    def getRequestId(self):
        return self.request_id

In [18]:
class Request():
    def __init__(self, request_id, origin, destination, time, passengerAmount):
        self.request_id = request_id
        self.origin = Request_Cords(origin.getLatitude(), origin.getLongitude(), self.request_id, True)
        self.destination = Request_Cords(destination.getLatitude(), destination.getLongitude(), self.request_id, False)
        self.time = time
        self.state = REQUESTSTATES["NOT_CALLED"]
        self.passengerAmount = passengerAmount

    def __str__(self):
        return "Request: id: {}, passengerAmount: {}, time: {}, origin: {}, destination: {},".format(self.request_id, self.passengerAmount, self.time, self.origin, self.destination)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.request_id == other.request_id

    #Getters
    def getId(self):
        return self.request_id

    def getPassengerAmount(self):
        return self.passengerAmount
    
    def getOrigin(self):
        return self.origin
    
    def getDestination(self):
        return self.destination
    
    def getTime(self):
        return self.timeOfRequest
    
    def getState(self):
        return self.state
    
    def changeState(self, state):
        self.state = REQUESTSTATES[state]

In [19]:
class Route:
    def __init__(self):
        self.routeList = RouteLinkedList()
        self.idCounter = 0 #Used to give each request pair a unique id that is small in memory size

    def __str__(self):
        return "{\n"+str(self.routeList) + "}, idCounter: " + str(self.idCounter)

    def __repr__(self):
        return self.__str__()
        
    def getRequestOSRMJSON(self, vehiclePosition, steps=False):
        if(steps == False):
            steps = "false"
        else:
            steps = "true"
        start_coords = str(vehiclePosition.getLongitude()) + "," + str(vehiclePosition.getLatitude())
        middle_coords = ""
        #Traverse the linked list to get the coordinates
        route = self.getRouteHead()
        if route == None:
            ValueError("The route is empty")
        while(route != None):
            middle_coords += str(route.cords.getLongitude()) + "," + str(route.cords.getLatitude()) + ";"
            route = route.next
        middle_coords = middle_coords[:-1]

        url = f"http://{OSRM_HOST}:{OSRM_PORT}/route/v1/driving/{start_coords};{middle_coords}?overview=false&steps={steps}"
        response = requests.get(url)
        # Parsing the response
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            Exception("Failed to get a response from the OSRM server")
    
    def optimizeRoute(self):
        pass

    def handleAddRequest(self, request):
        self.routeList.insertAtEnd(request.origin)
        self.routeList.insertAtEnd(request.destination)
        self.idCounter += 1
        self.optimizeRoute()
        return (self.idCounter-1) #Return the id of the request pair

    def handleNextArrival(self):
        cordPoint = self.routeList.deleteAtStart()
        return cordPoint
    
    def calculateTotalDistance(self, vehiclePosition):
        data = self.getRequestOSRMJSON(vehiclePosition)
        distance = data['routes'][0]['distance']  # Distance in meters
        return distance
    
    def calculateTotalTime(self, vehiclePosition):
        data = self.getRequestOSRMJSON(vehiclePosition)
        duration = data['routes'][0]['duration']  # Duration in seconds
        return duration
    
    def getListOfCords(self):
        cordsList = []
        route = self.getRouteHead()
        
        while(route != None): #Traverse the linked list to get the coordinates
            cordsList.append(route.cords)
            route = route.next

        return cordsList #If list is empty, [] is returned
    
    def getRouteHead(self):
        return self.routeList.head
    
    def getSize(self):
        return self.routeList.getSize() #Does not include vehicle position

In [20]:
class Vehicle():
    def __init__(self, vehicle_id, capacity, origin):
        self.vehicle_id = vehicle_id
        self.capacity = capacity
        self.currentCapacity = 0
        self.currentPosition = origin
        self.state = VEHICLESTATES["IDLE"]
        self.route = Route()
        self.onGoingRequestList = {}
        self.completedRequestList = {}

    def __str__(self):
        return "Vehicle: id: {}, currentPassengers: {}, route_size: {}, currentPosition: ({})".format(self.vehicle_id, self.currentCapacity, self.route.getSize(), self.currentPosition)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.vehicle_id == other.vehicle_id
    
    def addRequestToRoute(self, request):
        if request.getPassengerAmount() + self.currentCapacity > self.capacity: #Check Passengers Limit
            Exception("Request exceeds capacity of vehicle")
      
        self.route.handleAddRequest(request) #Add request to route
        requestId = request.getId()
        self.currentCapacity += request.getPassengerAmount() #Update the amount of passengers in the vehicle
        request.changeState("SCHEDULED") #Change the state
        print("Request id: ", requestId)
        self.onGoingRequestList[requestId] = request

    def arrivedAtNextStop(self):
        cords = self.route.handleNextArrival()
        requestId = cords.getRequestId()
        if cords.getStart() == True:
            print("HereFirst")
            self.onGoingRequestList[requestId].changeState("IN_PROGRESS") #Change the state
            #No need to add the passengers to the vehicle since they are acounted for in the addReuqestToRoute method
            self.currentPosition = Cords(cords.getLatitude(), cords.getLongitude()) #Update the position of the vehicle
        else:
            print("HereSecond")
            self.onGoingRequestList[requestId].changeState("COMPLETED") #Change the state
            self.completedRequestList[requestId] = self.onGoingRequestList[requestId] #Add the request to the completed list
            self.currentCapacity -= self.onGoingRequestList[requestId].getPassengerAmount() #Update the amount of passengers in the vehicle
            print(self.onGoingRequestList[requestId].getPassengerAmount())
            del self.onGoingRequestList[requestId] #Remove the request from the onGoing list
            self.currentPosition = Cords(cords.getLatitude(), cords.getLongitude()) #Update the position of the vehicle 
        return cords
    
    def printRoute(self):
        print("Vehile "+ str(self.vehicle_id)+" route: "+str(self.route))

    def getRouteDistance(self):
        return self.route.calculateTotalDistance(self.currentPosition)
    
    def getRouteTime(self):
        return self.route.calculateTotalTime(self.currentPosition)
    
    def getPosition(self):
        return self.currentPosition
    
    def getCurrentCapacity(self):
        return self.currentCapacity
    
    def getCapacity(self):
        return self.capacity
